In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs

In [90]:
df = pd.read_excel('Excel/Campeones.xlsx')
lista_horarios = list()
offset = 0
claves_excluidas = [1537]
open_link = False #bandera booleana para materias vinculadas (mismo grupo pero horario asimetrico)

In [111]:
for clave in claves_excluidas:
   df.drop(df[df['Clave']==clave].index, inplace=True)
materias = len(df.Clave.unique())
df.Clave.unique()
df


,Clave,Gpo,Profesor,Tipo,Horario,Días,Salón,Nombre,Cupo,Puntaje,...,Mar,Mie,Jue,Vie,Sab,Inicio,Fin,Inicio_min,Fin_min,Duracion
0,1645,5,M. I. MARIA DEL SOCORRO GUEVARA RODRIGUEZ,T,15:00 a 17:00,"Mar, Jue",J110,Dis. Dig. Mod,8,NaN,...,1,0,1,0,0,15:00,17:00,900,1020,2.0
1,1644,8,M.A. FRANCISCO JAVIER ROJAS DURAN,T,07:00 a 10:00,"Vie, Sab",B103,Bases de Datos,1,NaN,...,0,0,0,1,1,07:00,10:00,420,600,3.0
2,1644,1,ING. FERNANDO ARREOLA FRANCO,T,07:00 a 09:00,"Lun, Mie, Vie",A301,Bases de Datos,2,NaN,...,0,1,0,1,0,07:00,09:00,420,540,2.0
3,1644,3,ING. LUCILA PATRICIA ARELLANO MENDOZA,T,11:00 a 13:00,"Lun, Mie, Vie",B103,Bases de Datos,0,NaN,...,0,1,0,1,0,11:00,13:00,660,780,2.0
4,6645,2,M. I. MARIA DEL SOCORRO GUEVARA RODRIGUEZ,L+,15:00 a 17:00,Lun,T101,Lab. Dis. Dig. Mod,6,NaN,...,0,0,0,0,0,15:00,17:00,900,1020,2.0
5,1562,1,M.C. EDGAR BALDEMAR AGUADO CRUZ,T,17:30 a 19:00,"Lun, Mie",A306,Circuitos Electricos,0,NaN,...,0,1,0,0,0,17:30,19:00,1050,1140,1.5
6,6562,8,M.I. PATRICIA HONG CIRION,L+,20:00 a 22:00,Mar,P215,Lab. Circuitos Electricos,5,NaN,...,1,0,0,0,0,20:00,22:00,1200,1320,2.0
7,6562,7,M.I. JORGE ALBERTO UC MARTIN,L+,18:00 a 20:00,Mar,P215,Lab. Circuitos Electricos,4,NaN,...,1,0,0,0,0,18:00,20:00,1080,1200,2.0
8,1643,4,M.I. TANYA ITZEL ARTEAGA RICCI,T,17:00 a 19:00,"Mar, Jue",B204,Admin. Proy. Softw.,5,NaN,...,1,0,1,0,0,17:00,19:00,1020,1140,2.0
9,840,5,M.C. LAURA SANDOVAL MONTAÑO,T,11:00 a 13:00,"Mar, Jue",B304,Sistemas Operativos,3,NaN,...,1,0,1,0,0,11:00,13:00,660,780,2.0


In [92]:
def noHaySolape(horario,check_df,n):    
    """
    Verifica si existe un solape entre un potencial horario y una materia (renglón del dataframe)

    Parameters
    ----------
        horario: dataFrame
            Es el horario que se va armando
        check_df: DataFrame
            Es el df con las materias
        n: int
            Es el renglón actual
    """
    global open_link
    check = check_df.iloc[n]
    
    for i in range(0,len(horario)): 
        v = horario.iloc[i,9:15] * check.iloc[9:15] #Checa el vector de días de la semana y los multiplica dia a dia, si vale cero, entonces no comparten días
        if ((not v.equals(cero)) and (haySolapeHoras(horario.iloc[i],check))): #Si comparten días y ademas horas entonces sí hay solape
            return False

    #Cuando haya salido del for, significa que no hay ningún posible solape
    if(open_link == True): #Si la materia que se está comprobando esta linkeada
        #if isOpen_link(horario,check):
        if n<(len(df)-1):
            next = df.iloc[n+1]
            if not isOpen_link(horario,next):#Verifica si la materia que se está checando está vinculada a alguna materia del horario
                open_link = False #ya se ha pagado el vinculo que se debía
        else:
            open_link = False
            
    return True

In [93]:
#Verificación por horario
def haySolapeHoras(a,b): #DataFrame, comparativa a comparativa b
    if b.Inicio_min - a.Inicio_min == 0: #empiezan a la misma hora
        return True
    if b.Inicio_min - a.Inicio_min > 0: #b empieza después que a
        if(b.Inicio_min - a.Inicio_min < a.Duracion*60):
            return True
    else:
        if(abs(b.Inicio_min - a.Inicio_min) < b.Duracion*60):
            return True 
    return False
    

# Get dummies de los días

In [94]:
#orden = list(['Lun','Mar','Mie','Jue','Vie','Sab'])
dias = pd.Series(df.Días).str.get_dummies(sep = ', ')
orden = list(['Lun','Mar','Mie','Jue','Vie','Sab'])
dias[orden] = 0
dias = dias[orden]
dias += pd.Series(df.Días).str.get_dummies(sep = ', ')
dias.fillna(0,inplace=True,downcast = 'int64')

In [95]:
df = pd.concat([df,dias],axis=1)

# Get dummies de las horas

In [96]:
horas = df.Horario.str.split(" a ")

In [97]:
horas = pd.DataFrame.from_dict(dict(zip(horas.index, horas.values))).transpose()
horas.columns = ['Inicio','Fin']

In [98]:
df = pd.concat([df,horas],axis=1)


In [99]:
hrs_y_min = df.Inicio.str.split(":")
hrs_y_min.str[0]
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)
minutos
df = df.assign(Inicio_min = minutos)

hrs_y_min = df.Fin.str.split(":")
hrs_y_min.str[0]
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)
minutos
df = df.assign(Fin_min = minutos)
df

duracion = df.Fin_min - df.Inicio_min
df = df.assign(Duracion = duracion/60)


In [100]:
def generarHorarios():
    """
    Genera los horarios, realiza un arbol de posibilidades por cada opcion de la primer materia
    """
    n=0
    global open_link
    while(df.iloc[n].Clave == df.Clave.unique()[0]):
        open_link = False
        horario = pd.DataFrame(columns = df.columns) #crea un df con las mismas columnas para ir armando el horario
        combinarMaterias(df,horario,n)
        n = n+1

In [101]:
def combinarMaterias(df,horario,renglon_actual):
    """
    Es una función recursiva que toma los parámetros para ir concatenando un horario

    Parameters
    ------------
        df: dataFrame
            El dataframe que contiene todas las materias
        horario: dataFrame
            El horario que se va formando
        renglon_actual: int
            El indice del renglón del df que se concatena
    
    """
    global open_link
    if renglon_actual < len(df):
        new_row = df.copy()[df.index == renglon_actual]
        horario = pd.concat([horario, new_row])         #Copia el renglon actual al horario
        for i in range(renglon_actual+1,len(df)):
            if hayVinculoAtras(df,i):
                if not includes(horario,df.iloc[i-1]): #Esta vinculado con una rama anterior
                    continue

            if isClaveRepetida(horario,df.iloc[i]): 
                if not isOpen_link(horario,df.iloc[i]): #Si la materia es la misma pero con grupo diferente, la omite
                    continue
                #else la materia esta linkeada
            else: #Se tiene una materia diferente
                if open_link == True:
                    continue
                
            #isOpen_link(horario,df.iloc[i])
            temp = open_link
            if noHaySolape(horario,df,i):
                combinarMaterias(df,horario,i) #Al ser recursiva, va concatenando las filas del excel que no se solapen y sean diferentes materias
                open_link = temp
                #isOpen_link(horario,df.iloc[i]) #Recupera el estado de la bander open_link para eliminar opciones
        
        if(len(horario.Clave.unique()) == materias and open_link == False): #Comprueba la variable horarios llenos para ver si tiene que cumplir con cubrir todas las materias del excel y no haya una incompleta
                lista_horarios.append(horario)
                plotHorario(horario,len(lista_horarios))
       

In [102]:
def includes(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [103]:
def hayVinculoAtras(df,n):
    if(n>1):
        if (df.iloc[n].Gpo == df.iloc[n-1].Gpo and df.iloc[n].Clave == df.iloc[n-1].Clave):
            return True
    return False

In [104]:
def isClaveRepetida(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [105]:
def isOpen_link(horario,materia):
    """
    Cambia la bandera open_link si la materia está vinculada con alguna del horario
    Una materia está vinculada con el horario si tiene misma clave y grupo.

    Parameters
    ---------
        horario: DataFrame
            Es el horario que se está armando
        materia: Series
            Es una materia con la que se está comprobando
    """
    global open_link
    for i in range(0,len(horario)): #Checa con todas las materias
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            open_link = True #Se vinculan únicamente si coincide alguna materia en grupo y clave
            return True
    return False
    

In [106]:
def plotMateria(df,gnt):
    import random
    R = random.uniform(0.3,0.75)
    G = random.uniform(0, 0.1)
    B = random.uniform(0.6, 1)
    
    i = 0
    hora = (df.Inicio_min/60)
    duracion = df.Duracion
    
    for cat in df.iloc[9:15]: #Lo que hace es particionar el df a los días de la semana
        if cat == 1:
            gnt.broken_barh([(i,1)], (hora, duracion) ,facecolors = (R, G, B))
            nombre_materia = df.Nombre[:10]                                                     #Corta el texto para que quepa en el cuadro
            nombre_profe = df.Profesor.split(" ")
            nombre_pila = nombre_profe[1]
            nombre_pila = nombre_pila[:5]
            apellido = nombre_profe[-2]
            #nombre_pila = df.Profesor[:7]
            inicial = apellido [:1]
            nombre = nombre_materia + "\n" + str(df.Gpo) + " " + nombre_pila  + " " + inicial                                    
            gnt.text(i+0.1,hora+1.5,nombre,color="white",fontweight = "bold",fontsize = "small") #x,y,texto,color,grosor y tamaño
        i = i+1


In [107]:
def plotHorario(horario, numero):
    import matplotlib.pyplot as plt
    fig, gnt = plt.subplots()
    gnt.set_ylim(6,23 )
    gnt.set_xlim(0,6)
    gnt.set_xlabel('Día')
    gnt.set_ylabel('Horas')

    #horas = ['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']
    dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado']
    plt.gca().invert_yaxis()
    gnt.set_yticks(np.arange(7, 23,1))
    gnt.set_xticks(np.arange(0,6,1))
    gnt.set_xticklabels(dias,ha="left")
    gnt.grid(True)

    for i in range(0,len(horario)):
        plotMateria(horario.iloc[i],gnt)

    plt.savefig("Horarios/opción" + str(numero) + ".png")
    plt.close()

In [108]:
def clearCarpeta():
    import os, shutil
    folder = 'Horarios/'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('No fue posible borrar la carpeta %s. Excepción: %s' % (file_path, e))
    

In [109]:
def imprimirHorarios():
    clearCarpeta()
    if len(lista_horarios) == 0:
        print("No se puede generar ningún horario con las materias ingresadas. POSIBLE SOLAPE")
        return 0
    n=1
    for horario in lista_horarios:
        plotHorario(horario,n)
        #print(horario.iloc[:,0:8].drop(['Tipo','Cupo'],axis=1))
        horario.drop(horario.tail(1).index,inplace=True)
        n+=1
    

In [110]:
horario = pd.DataFrame(columns = df.columns)
horario
cero = pd.Series([0,0,0,0,0,0], index = orden).astype(object)
generarHorarios()
print("Horarios generados: " + str(len(lista_horarios)))
imprimirHorarios()


Horarios generados: 0
No se puede generar ningún horario con las materias ingresadas. POSIBLE SOLAPE


0